In [1]:
import pandas as pd
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from scipy.optimize import minimize
import statsmodels.api as sm
import numpy as np 
from sklearn.model_selection import train_test_split as tts, KFold, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

### Import the provided dataset “AirI.csv.” The first column represents the response variable, and the rest are the predictors (input features). 

In [2]:
data = pd.read_csv('/Users/rebeccawagner/Documents/GitHub/Data 441/Data/AirI.csv',names=[i for i in range(-1,511)])
data.head()

,-1,0,1,2,3,4,5,6,7,8,...,501,502,503,504,505,506,507,508,509,510
0,3.4482,0.001314,0.15747,0.688300,0.152920,0.00000,0.00000,0.096603,0.52135,0.38205,...,-2.94590,0.219960,-0.69045,0.24044,-0.258640,1.33870,0.114820,-0.43046,0.82309,1.54350
1,3.3019,0.188490,0.55319,0.258290,0.000031,0.00000,0.00000,0.039008,0.43211,0.52716,...,-0.22680,-0.428570,0.74992,-0.11192,1.583000,0.88565,-0.053227,0.67796,-1.72010,-0.55051
2,2.2894,0.052169,0.43852,0.494030,0.015282,0.00000,0.00000,0.000000,0.00000,0.39484,...,1.15790,0.010118,0.73593,1.35600,0.057821,-1.44140,1.581800,1.25420,1.02540,-1.01890
3,2.6207,0.000000,0.00000,0.004213,0.109340,0.49344,0.39300,0.000000,0.00000,0.57854,...,-0.69767,1.577800,0.54899,-0.78042,1.136700,-0.84240,0.993120,-0.44576,-0.78778,-1.49090
4,2.8439,0.000000,0.00000,0.019504,0.260470,0.54986,0.17016,0.010568,0.30520,0.67046,...,-1.49170,-0.688640,-0.69924,0.36074,-0.685860,-0.76161,-0.354560,0.83093,-0.27100,0.38861


In [3]:
y = data[-1].values
x = data.loc[:,0:511].values

In [4]:
scale = StandardScaler()
x = scale.fit_transform(x)

In [5]:
x = np.column_stack([np.ones(len(x)),x])

### Implement with a SKLearn wrapper the Square Root Lasso and Smoothly Clipped Absolute Deviations algorithms.

#### Square Root Lasso

In [6]:
class SQRTLasso:
    def __init__(self, alpha = 0.5,intercept=True):
        self.alpha = alpha
        self.intercept = intercept
    
    def fit(self, x, y):
        alpha = self.alpha
        intercept = self.intercept
        self.xtrain_ = x
        self.ytrain_ = y
        self.fitted_model_ = sm.OLS(y,x).fit_regularized(method='sqrt_lasso', alpha=alpha)
        self.coef_ = self.fitted_model_.params
    
    def predict(self, x_new):
        check_is_fitted(self)
        x = self.xtrain_
        y = self.ytrain_
        alpha = self.alpha
        intercept = self.intercept
        fitted_model = self.fitted_model_
        return  fitted_model.predict(x_new)

    def get_params(self, deep=True):
    # suppose this estimator has parameters "f", "iter" and "intercept"
        return {"alpha": self.alpha,"intercept":self.intercept}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

#### SCAD

In [167]:
def scad_penalty(beta_hat, lambda_val, a_val):
    is_linear = (np.abs(beta_hat) <= lambda_val)
    is_quadratic = np.logical_and(lambda_val < np.abs(beta_hat), np.abs(beta_hat) <= a_val * lambda_val)
    is_constant = (a_val * lambda_val) < np.abs(beta_hat)
    
    linear_part = lambda_val * np.abs(beta_hat) * is_linear
    quadratic_part = (2 * a_val * lambda_val * np.abs(beta_hat) - beta_hat**2 - lambda_val**2) / (2 * (a_val - 1)) * is_quadratic
    constant_part = (lambda_val**2 * (a_val + 1)) / 2 * is_constant
    return linear_part + quadratic_part + constant_part
    
def scad_derivative(beta_hat, lambda_val, a_val):
        return lambda_val * ((beta_hat <= lambda_val) + (a_val * lambda_val - beta_hat)*((a_val * lambda_val - beta_hat) > 0) / ((a_val - 1) * lambda_val) * (beta_hat > lambda_val))

def scad(beta):
  n = len(y)
  errors = y - x.dot(beta)
  return 1/n*np.sum(errors**2) + np.sum(scad_penalty(beta,lam,a))
  
def dscad(beta):
  errors = y - x.dot(beta)
  n = len(y)
  return -2/n*errors.dot(x)+scad_derivative(beta,lam,a)


In [199]:
class SCAD_C:

    def __init__(self, alpha, lam):
        self.alpha = alpha
        self.lam = lam 

    def fit(self, x, y):

        alpha = self.alpha
        lam = self.lam

        beta_0 = np.random.uniform(size=x.shape[1])
        output = minimize(scad, beta_0, method='L-BFGS-B', jac=dscad,options={'gtol': 1e-8, 'maxiter': 50000,'maxls': 25,'disp': True})
        self.coef_ = output.x

    def predict(self, x):
        check_is_fitted(self)
        beta_hat = self.coef_
        y_hat = np.dot(x,beta_hat)
        return y_hat

### Test Lasso & SCAD

In [74]:
xtrain, xtest, ytrain, ytest = tts(x, y, test_size = .3)

In [206]:
scad_c = SCAD_C(1.1, .001)
scad_c.fit(xtrain,ytrain)
scad_c.predict(x=xtest)

array([2.84366718, 2.00102084, 4.59910813, 3.73147479, 3.6588353 ,
       3.53080779, 4.34360318, 3.68813438, 3.3319936 , 2.2224287 ,
       0.99938763, 2.75588531, 2.35233345, 2.7165324 , 2.52113219,
       2.84376186, 3.29904788, 2.52072185, 4.17741276, 2.88615532,
       1.91400012, 2.35107888, 3.03547995, 2.28750394, 2.88798348,
       2.22284521, 5.51540687, 4.39925498, 2.08101852, 3.23668608,
       4.02227046, 3.14085972, 2.35047438, 2.84206867])

In [207]:
lasso = SQRTLasso()
lasso.fit(xtrain,ytrain)
lasso.predict(xtest)

array([2.96647016, 2.24163677, 3.97716202, 3.21443781, 4.23756744,
       3.02410755, 4.57889547, 3.59637893, 2.77408521, 1.8515914 ,
       2.17731132, 2.99264712, 2.59257639, 3.14194203, 3.67861907,
       2.66939248, 2.92044772, 3.75799873, 4.95433975, 2.70906659,
       2.696711  , 2.3735712 , 3.48509357, 2.04703783, 2.88072014,
       2.74533089, 3.96767664, 4.77367921, 2.74950942, 3.11483317,
       3.04375637, 2.66510645, 2.93859052, 2.4458948 ])

### Determine the best choice of hyperparameters for the two methods (within finite ranges) by using GridsearchCV

In [208]:
params = {'alpha': [i for i in range(1,25)]}

gs_lasso = GridSearchCV(SQRTLasso(),
                      param_grid=params,
                      scoring='neg_mean_squared_error',
                      cv=5)

gs_lasso.fit(x,y)
gs_lasso.best_params_

{'alpha': 2}

In [ ]:
params = {'alpha': [i for i in np.linspace(1,10,20)],
          'lam': [i for i in np.linspace(1.10,100)]}

gs_scad = GridSearchCV(SCAD_C(),
            param_grid = params,
            scoring='neg_mean_squared_error',
            cv=5)

gs_scad.fit(x,y)

### Do the same with ElasticNet

In [215]:
params = [{'alpha': [i for i in np.linspace(1,10,20)],
           'l1_ratio': [i for i in np.linspace(0.01,1,99)]}]

gs_elasticnet = GridSearchCV(ElasticNet(),
                      param_grid=params,
                      scoring='neg_mean_squared_error',
                      cv=5)

gs_elasticnet.fit(x,y)
gs_elasticnet.best_params_

{'alpha': 1.0, 'l1_ratio': 0.01}

### Based on the best set of hyperparameters for each method, find the 10-fold cross-validated prediction error and (on average) the number of variables selected in each case.

In [247]:
def Run_KFold(model, x, y):

    kf = KFold(n_splits = 10, shuffle=True) # set KFOLD
    mse_list = [] # initialize MSE list
    coef_list = []

    for idxtrain, idxtest in kf.split(x): # run for each of the splits
        xtrain = x[idxtrain]
        ytrain = y[idxtrain]
        ytest = y[idxtest]
        xtest = x[idxtest]

        model.fit(xtrain,ytrain)
        y_pred = model.predict(xtest)

        coef_list.append(model.coef_)
        mse_list.append(mse(ytest, y_pred))
    
    return mse_list, coef_list

In [250]:
lasso_mse, lasso_coef = Run_KFold(SQRTLasso(2),x,y)

In [249]:
Run_KFold(ElasticNet(alpha=1,l1_ratio=.01),x,y)

([0.6920833905737566,
  0.32287302985627503,
  0.23430699434792404,
  0.2622089186025035,
  0.4957780404118414,
  0.27918825472066955,
  0.3202769976887984,
  0.06128776947430526,
  0.1151284012997647,
  0.157923337821589],
 [array([ 0.00000000e+00, -1.82359168e-02, -0.00000000e+00,  2.43560827e-02,
          2.16186301e-02,  5.45467134e-03, -9.75804302e-03,  3.70660459e-02,
          1.77693436e-02, -1.48989885e-02, -1.81737810e-02, -1.81444071e-02,
         -0.00000000e+00, -2.24653460e-02, -2.83787950e-02, -9.60309518e-03,
          3.69015330e-02,  5.70409938e-02,  1.76006438e-02,  0.00000000e+00,
          1.43908714e-02, -0.00000000e+00, -0.00000000e+00, -1.45299910e-02,
         -9.32405254e-03,  1.49704650e-03,  1.07583894e-02,  0.00000000e+00,
          3.78837941e-02,  2.41453189e-02,  1.47815888e-02, -1.47873453e-02,
         -2.63044781e-02, -1.05413358e-02,  2.23601608e-02,  3.50801800e-03,
         -0.00000000e+00,  1.70024200e-02,  1.13498475e-02, -6.47136228e-04,
      